In [ ]:
import os
import dask
from dask.distributed import Client
from azureml.core import Run

In [ ]:
def update_packages():
    os.system('pip install --upgrade fsspec adlfs')

In [ ]:
computes = [dask.delayed(update_packages)() for i in range(10)]
computes

In [ ]:
run = Run.get_context()
run

c = Client(f'{run.get_metrics()["scheduler"]}')
c.compute(computes)
c

### Connect to cluster

In [1]:
import os
import dask
import time
import joblib
import fsspec
import socket
import matplotlib

import pandas as pd
import dask.dataframe as dd
import matplotlib.pyplot as plt

from datetime import datetime
from dask.distributed import Client
from IPython.core.display import HTML
from dask_ml.xgboost import XGBRegressor

from azureml.core import Run

%matplotlib inline

ImportError: libcuda.so.1: cannot open shared object file: No such file or directory

In [ ]:
run = Run.get_context()
run

In [ ]:
c = Client(f'{run.get_metrics()["scheduler"]}')
c

In [ ]:
dstore = run.experiment.workspace.datastores['data4dask']

In [ ]:
STORAGE_OPTIONS = {
    'account_name': dstore.account_name, 
    'account_key' : dstore.account_key
}

In [ ]:
protocol  = 'abfs'      # use 'adl' for Azure Data Lake Gen 1
container = 'datasets'  # only contains ISD, GFS is wip

In [ ]:
fs = fsspec.filesystem(protocol, **STORAGE_OPTIONS, container_name=container)

In [ ]:
fs.ls('/noaa/gfs')

In [ ]:
files = []
for file in fs.glob('noaa/gfs/GFSProcessed/year=*/month=*'): 
    files += fs.ls(f'{file}/')
files[-5:]

In [ ]:
# this is slow - there are ~50k files
files2 = []
for file in files: 
    files2 += fs.glob(f'{file}/*.parquet')
files2[-5:]

In [ ]:
files = files2

In [ ]:
len(files) # number of parquet files - there are other random files in there 

In [ ]:
df = dask.delayed(dd.read_parquet)(files, engine='pyarrow', storage_options=STORAGE_OPTIONS).compute()
%time df.head()

In [ ]:
df = df.set_index(dd.to_datetime(df.datetime).dt.floor('d'), sorted=False)
df = df.persist() 
%time len(df)

In [ ]:
%time len(df)

In [ ]:
%time df.describe().compute()

In [ ]:
%time bites = df.memory_usage(index=True, deep=True).sum().compute()
print(f'Dataframe is: {round(bites/1e9, 2)}GB')

In [ ]:
%time means = df.groupby(df.index).mean().compute()
means.head()

In [ ]:
for col in list(means.columns):
    fig = plt.figure(figsize=(16, 8))
    #plt.style.use('dark_background')
    means[col].plot(color='b')
    plt.title('Average of {}'.format(col))
    plt.xlim([datetime(2008, 1, 1), datetime(2018, 12, 31)])
    plt.grid()
    
    # optionally, log the image to the run
    run.log_image(f'mean_{col}', plot=plt)

You can see the images logged to the run in the studio.

In [ ]:
run

## Prepare data

In [ ]:
## insert any Pandas-like Dask data prep code 
df['temperature'] = df['temperature']*(9/5)+32 # 'Merica

## Write data

**Important:** you will have not access to write to this account, get your own storage account! 

See the [API documentation for dask.dataframe.to_parquet](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.to_parquet) for additional settings. 

In [ ]:
df = df.repartition(npartitions=df.npartitions*10)
%time dask.delayed(df.to_parquet)(f'abfs://outputs/noaa/isd_out.parquet', compression='lz4', storage_options=STORAGE_OPTIONS).compute()

![Write gif](media/write.gif)

## Train XGBoost model

In [ ]:
df = dask.delayed(dd.read_parquet)(files, engine='pyarrow', storage_options=STORAGE_OPTIONS).compute()

In [ ]:
files = []
for file in fs.glob('noaa/isd/year=*/month=*'): # see https://github.com/dask/adlfs/issues/34
    files += fs.ls(f'{file}/')
files = [f'{protocol}://{container}/{file}' for file in files if '2019' in file] 

df2019 = dask.delayed(dd.read_parquet)(files, engine='pyarrow', storage_options=STORAGE_OPTIONS).compute() 

In [ ]:
# begin data prep
df = df.fillna(0) 
df2019 = df2019.fillna(0) 

In [ ]:
df['month'] = df['datetime'].dt.month
df2019['month'] = df2019['datetime'].dt.month

In [ ]:
cols = list(df.columns)
cols = [col for col in cols if df.dtypes[col] != 'object' and col not in ['version', 'datetime']]
cols

In [ ]:
X = df[[col for col in cols if col not in ['temperature']]].persist()
y = df.temperature.persist()
# end data prep - persist intelligently per https://docs.dask.org/en/latest/best-practices.html

In [ ]:
xgb = XGBRegressor(n_estimators=16)
%time xgb.fit(X, y)

In [ ]:
%time y_pred = xgb.predict(X).compute()

In [ ]:
rmse = (((y.to_dask_array().compute()-y_pred)**2).mean())**.5 # runs locally, distribute (?)
print(f'Training RMSE: {round(rmse, 3)}')

## Evaluate model

In [ ]:
X_test = df2019[[col for col in cols if col not in ['temperature']]].persist()
y_test = df2019.temperature.persist()

In [ ]:
%time y_pred = xgb.predict(X_test).compute()

In [ ]:
rmse = (((y_test.to_dask_array().compute()-y_pred)**2).mean())**.5 # runs locally, distribute (?)
print(f'Test RMSE: {round(rmse, 3)}')

## Register model

In [ ]:
model_path = 'xgboost_noaa_isd.joblib.dat'
joblib.dump(xgb, model_path)
xgb = joblib.load(model_path)

In [ ]:
model = Model.register(ws, model_path, 'xgboost-noaa-isd', 
                       description='Dask XGBoost NOAA ISD temperature predictor',
                       model_framework='XGBoost')

## End the run

Cluster will return to 0 nodes.

In [ ]:
c.close()
run.cancel()

In [ ]:
t_end = time.time()
print(f'Total run time: {round((t_end-t_start)/60, 2)} minutes')